<a href="https://colab.research.google.com/github/Aakash326/ML-models/blob/main/AutoAgent_Cmo_brandmarketer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q pyautogen openai python-dotenv gradio google-generativeai "ag2[gemini]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.4/101.4 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 859.2/859.2 kB 23.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.6/65.6 kB 4.4 MB/s eta 0:00:00


In [2]:
import warnings
warnings.filterwarnings("ignore")

# Import necessary libraries
import os
import autogen
import gradio as gr
from openai import OpenAI  # Keep for reference if needed
from dotenv import load_dotenv
from IPython.display import display, Markdown
import random  # Used later for unique Gradio outputs
import google.generativeai as genai  # Import the Google library

# Load environment variables from the .env file
load_dotenv()

# Retrieve API keys from environment variables
openai_api_key = os.getenv("OPENAI_API_KEY")
google_api_key = os.getenv("GOOGLE_API_KEY")  # Load the Google API key

print("Setup Complete: Libraries installed and API keys loaded.")

Setup Complete: Libraries installed and API keys loaded.


In [3]:
from google.colab import userdata
from openai import OpenAI

# ✅ Load the secret from Colab
openai_api_key = userdata.get("OPENAI_API_KEY")  # must match exactly what you named it in the Secrets panel

# ✅ Initialize the OpenAI client
client = OpenAI(api_key=openai_api_key)

In [4]:
gemini_ai = userdata.get("GEMINI_API_KEY")
# Initialize the model
genai.configure(api_key=gemini_ai)
gemini_model = genai.GenerativeModel("gemini-2.0-flash")

In [5]:
def print_markdown(text):
    display(Markdown(text))

In [6]:
config_list_openai = [
    {
        "model": "gpt-4o-mini",
        "api_key": openai_api_key,
    }
]

llm_config_openai = {
    "config_list": config_list_openai,
    "temperature": 0.7,  # Use a slightly higher temp for creative marketing ideas
    "timeout": 120,
}

In [7]:
cmo_prompt = """You are the Chief Marketing Officer (CMO) of a new shoe brand (sustainable).
Provide high-level strategy, define target audiences, and guide the Marketer. Focus on the big picture. Be concise."""

In [8]:
brand_marketer_prompt = """You are the Brand Marketer for the shoe brand. Brainstorm creative, specific campaign ideas (digital, content, experiences).
Focus on tactics and details. Suggest KPIs for your ideas."""

In [9]:
cmo_agent_openai = autogen.ConversableAgent(
    name = "Chief_Marketing_Officer_OpenAI",
    system_message = cmo_prompt,
    llm_config = llm_config_openai,  # Assign the OpenAI config
    human_input_mode = "NEVER")

print(f"Agent '{cmo_agent_openai.name}' created (using OpenAI).")

Agent 'Chief_Marketing_Officer_OpenAI' created (using OpenAI).


In [10]:
brand_marketer_agent_openai = autogen.ConversableAgent(
    name = "Brand_Marketer_OpenAI",
    system_message = brand_marketer_prompt,
    llm_config = llm_config_openai,  # Assign the same OpenAI config
    human_input_mode = "NEVER")

print(f"Agent '{brand_marketer_agent_openai.name}' created (using OpenAI).")

Agent 'Brand_Marketer_OpenAI' created (using OpenAI).


In [11]:
initial_task_message = """
Context: We're launching a new sustainable shoe line and need campaign ideas
Instruction: Brainstorm a campaign concept with specific elements
Input: Our sustainable, futuristic shoe brand needs marketing direction
Output: A concise campaign concept with the following structure:
Brand Marketer, let's brainstorm initial campaign ideas for our new sustainable shoe line.
Give me a distinct campaign concept. Outline: core idea, target audience, primary channels, and 1-2 KPIs. Keep it concise. Try to arrive at a final answer in 2-3 turns.
"""

print("--- Starting Agent Conversation (OpenAI Only) ---")
print("Chief Marketing Officer (OpenAI) initiating chat with Brand Marketer (OpenAI). Max Turns = 4")
print("--------------------------------------------------")

# Chief Marketing Officer (OpenAI) initiates the chat with Brand Marketer (OpenAI)
chat_result_openai_only = cmo_agent_openai.initiate_chat(
    recipient = brand_marketer_agent_openai, message = initial_task_message, max_turns = 4
)

print("--------------------------------------------------")
print("--- Conversation Ended (OpenAI Only) ---")

--- Starting Agent Conversation (OpenAI Only) ---
Chief Marketing Officer (OpenAI) initiating chat with Brand Marketer (OpenAI). Max Turns = 4
--------------------------------------------------
Chief_Marketing_Officer_OpenAI (to Brand_Marketer_OpenAI):


Context: We're launching a new sustainable shoe line and need campaign ideas
Instruction: Brainstorm a campaign concept with specific elements
Input: Our sustainable, futuristic shoe brand needs marketing direction
Output: A concise campaign concept with the following structure:
Brand Marketer, let's brainstorm initial campaign ideas for our new sustainable shoe line.
Give me a distinct campaign concept. Outline: core idea, target audience, primary channels, and 1-2 KPIs. Keep it concise. Try to arrive at a final answer in 2-3 turns.


--------------------------------------------------------------------------------
Brand_Marketer_OpenAI (to Chief_Marketing_Officer_OpenAI):

### Campaign Concept: "Step Into the Future"

**Core Idea:**  

In [12]:
def print_chat_history(chat_result):
    """Any chat result object has a chat_history attribute that contains the conversation history.
    This function prints the conversation history in a readable format.
    """
    for i in chat_result.chat_history:
        print_markdown(i['name'])
        print("_"*100)
        print_markdown(i['content'])
        print("_"*100)

In [13]:
print_chat_history(chat_result_openai_only)

Chief_Marketing_Officer_OpenAI

____________________________________________________________________________________________________



Context: We're launching a new sustainable shoe line and need campaign ideas
Instruction: Brainstorm a campaign concept with specific elements
Input: Our sustainable, futuristic shoe brand needs marketing direction
Output: A concise campaign concept with the following structure:
Brand Marketer, let's brainstorm initial campaign ideas for our new sustainable shoe line.
Give me a distinct campaign concept. Outline: core idea, target audience, primary channels, and 1-2 KPIs. Keep it concise. Try to arrive at a final answer in 2-3 turns.


____________________________________________________________________________________________________


Brand_Marketer_OpenAI

...

____________________________________________________________________________________________________


Brand_Marketer_OpenAI

____________________________________________________________________________________________________


### Finalized Campaign Concept: "Step Into the Future"

**Core Idea:**  
The "Step Into the Future" campaign aims to create an emotional connection between consumers and our sustainable shoe line through an immersive digital experience. By utilizing a virtual reality (VR) platform, users can explore the eco-friendly lifecycle of our shoes, witnessing the production process and understanding the positive environmental impact of their choices. Additionally, an augmented reality (AR) app will allow consumers to visualize how our shoes fit into their lifestyles, reinforcing a commitment to sustainability.

**Target Audience:**  
Our primary audience consists of eco-conscious consumers aged 18-35, particularly urban millennials and Gen Z, who prioritize sustainability and are tech-savvy. This demographic is passionate about brands that align with their values and is likely to share their experiences on social media, amplifying our campaign's reach.

**Primary Channels:**  
1. **Social Media:** Use Instagram and TikTok for engaging content, including teaser videos, interactive polls, and user-generated content challenges that encourage followers to share their sustainable practices and experiences with our products.
2. **Website:** Develop a dedicated campaign page that features the VR experience, AR app access, and informative articles on sustainability, alongside a seamless e-commerce platform for purchasing shoes.
3. **Influencer Collaborations:** Partner with eco-conscious influencers to authentically share their experiences with the VR/AR tools and our shoes, leveraging their credibility to enhance brand visibility.

**KPIs:**
1. **Engagement Rate:** Aim for a 15% increase in social media interactions, measured by likes, comments, shares, and participation in user-generated content.
2. **AR/VR Experience Usage:** Target 15,000 unique users engaging with the AR app and VR experience within the first month, assessing the depth of consumer engagement with our brand narrative and products.

This campaign positions our brand as a pioneering leader in sustainable footwear, using innovative technology to effectively engage and inspire our target audience. By emphasizing both the innovative aspects of our shoes and the significance of sustainable living, we aim to cultivate a community around eco-friendly practices and foster lasting brand loyalty.

____________________________________________________________________________________________________


In [19]:
user_proxy_agent = autogen.UserProxyAgent(
    name = "Human_User_Proxy",
    human_input_mode = "ALWAYS",  # Prompt user for input until 'exit'
    max_consecutive_auto_reply = 1,
    is_termination_msg = lambda x: x.get("content", "").rstrip().lower() in ["exit", "quit", "terminate"],
    code_execution_config = False,
    system_message = "You are the human user interacting with a multi-model AI team (Gemini CMO, OpenAI Marketer). Guide the brainstorm. Type 'exit' to end.",
)

print(f"Agent '{user_proxy_agent.name}' created for HIL with multi-model team.")

# %% [markdown]
# **Initiating HIL Chat with the Multi-Model Team**

# %%
print("--- Starting Human-in-the-Loop (HIL) Conversation (Multi-Model) ---")
print("You will interact with Gemini CMO and OpenAI Marketer. Type 'exit' to end.")
print("---------------------------------------------------------------------")

Agent 'Human_User_Proxy' created for HIL with multi-model team.
--- Starting Human-in-the-Loop (HIL) Conversation (Multi-Model) ---
You will interact with Gemini CMO and OpenAI Marketer. Type 'exit' to end.
---------------------------------------------------------------------


In [20]:
cmo_agent_openai.reset()  # Reset Gemini CMO
brand_marketer_agent_openai.reset()  # Reset OpenAI Marketer
user_proxy_agent.reset()

In [21]:
from autogen import GroupChat, GroupChatManager

# Create a GroupChat with multiple agents
# This sets up a collaborative chat environment where multiple agents can interact
groupchat = GroupChat(
    agents = [user_proxy_agent, cmo_agent_openai, brand_marketer_agent_openai],  # List of agents participating in the group chat
    messages = [ ],  # Initialize with empty message history
    max_round = 20,  # Optional: Limits how many conversation rounds can occur before terminating
)

# Create a manager for the group chat
# The GroupChatManager orchestrates the conversation flow between agents
# It determines which agent should speak next and handles the overall conversation logic
group_manager = GroupChatManager(groupchat = groupchat, llm_config = llm_config_openai)

In [22]:
group_chat_result = group_manager.initiate_chat(
    recipient = user_proxy_agent,  # Start by talking to the Gemini CMO
    message = """Hello team!!""",
)

print("---------------------------------------------------------------------")
print("--- Conversation Ended (Human terminated or Max Turns) ---")

chat_manager (to Human_User_Proxy):

Hello team!!

--------------------------------------------------------------------------------
Replying as Human_User_Proxy. Provide feedback to chat_manager. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: hello team lets do this
Human_User_Proxy (to chat_manager):

hello team lets do this

--------------------------------------------------------------------------------

Next speaker: Chief_Marketing_Officer_OpenAI

Chief_Marketing_Officer_OpenAI (to chat_manager):

### High-Level Marketing Strategy for Sustainable Shoe Brand

**Vision:** Position the brand as the leader in sustainable footwear, emphasizing eco-friendliness, ethical production, and stylish design.

**Key Objectives:**
1. Build brand awareness and recognition within the sustainable fashion space.
2. Drive online and offline sales through targeted campaigns.
3. Cultivate a loyal customer base that advocates for sustainability.

### Target Audiences

1.

In [23]:
print_chat_history(group_chat_result)

chat_manager

____________________________________________________________________________________________________


Hello team!!

____________________________________________________________________________________________________


Human_User_Proxy

...

____________________________________________________________________________________________________


Human_User_Proxy

____________________________________________________________________________________________________


skip

____________________________________________________________________________________________________


In [14]:
config_list_gemini = [
    {
        "model": "gemini-2.0-flash",  # Or "gemini-pro"
        "api_key": google_api_key,
        "api_type": "google",  # Specify the API type for Autogen's Google integration
    }
]

llm_config_gemini = {
    "config_list": config_list_gemini,
    "temperature": 0.6,  # Maybe slightly less randomness for strategic Chief Marketing Officer
    "timeout": 120,
}

In [15]:
config_list_kimi=[
    {
        'model':'moonshotai/kimi-k2:free',
        'api_key': "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",
        'api_type': 'openai',
    }
]
llm_config_kimi = {
    "config_list": config_list_kimi,
    "temperature": 0.6,  # Maybe slightly less randomness for strategic Chief Marketing Officer
    "timeout": 120,
}

In [18]:
from openai import OpenAI

client = OpenAI(
  base_url="https://openrouter.ai/api/v1",
  api_key="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",
)

completion = client.chat.completions.create(
  extra_headers={
    "HTTP-Referer": "<YOUR_SITE_URL>", # Optional. Site URL for rankings on openrouter.ai.
    "X-Title": "<YOUR_SITE_NAME>", # Optional. Site title for rankings on openrouter.ai.
  },
  extra_body={},
  model="moonshotai/kimi-k2:free",
  messages=[
    {
      "role": "user",
      "content": "hi"
    }
  ]
)
print(completion.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'Provider returned error', 'code': 429, 'metadata': {'raw': 'moonshotai/kimi-k2:free is temporarily rate-limited upstream. Please retry shortly, or add your own key to accumulate your rate limits: https://openrouter.ai/settings/integrations', 'provider_name': 'Parasail'}}, 'user_id': 'user_305j67qCagGqaD5pqd2Y6Mtpp3N'}

In [17]:
# cmo_agent_gemini = autogen.ConversableAgent(
#     name = "Chief_Marketing_Officer_Gemini",
#     system_message = cmo_prompt,
#     llm_config = llm_config_gimini,  # Assign the Gemini config!
#     human_input_mode = "NEVER")

cmo_agent_kimi = autogen.ConversableAgent(
    name = "Chief_Marketing_Officer_Gemini",
    system_message = cmo_prompt,
    llm_config = llm_config_kimi,  # Assign the Gemini config!
    human_input_mode = "NEVER")

# Create the Brand Marketer Agent using OpenAI GPT (this is similar to before!)
# We can reuse the llm_config_openai defined earlier
brand_marketer_agent_openai_mixed = autogen.ConversableAgent(
    name = "Brand_Marketer_OpenAI",  # Keep name consistent if desired, or update
    system_message = brand_marketer_prompt,
    llm_config = llm_config_openai,  # Assign the OpenAI config!
    human_input_mode = "NEVER")

print(f"Agent '{cmo_agent_gemini.name}' created (using Google Gemini).")
print(f"Agent '{brand_marketer_agent_openai_mixed.name}' created (using OpenAI).")

NameError: name 'cmo_agent_gemini' is not defined

In [ ]:
import vertexai
# from langchain_google_vertexai import ChatVertexAI # No longer needed

# Initialize Vertex AI
vertexai.init(project="n8n-464215", location="us-central1")

# Now proceed with the rest of your agent setup

In [ ]:
print("--- Starting Agent Conversation (Multi-Model: Gemini + OpenAI) ---")
print("Chief Marketing Officer (Gemini) initiating chat with Brand Marketer (OpenAI). Max Turns = 4")
print("------------------------------------------------------------------")

# Chief Marketing Officer (Gemini) initiates the chat with the Brand Marketer (OpenAI)
chat_result_multi_model = cmo_agent_kimi.initiate_chat(
    recipient = brand_marketer_agent_openai_mixed,  # Target the OpenAI marketer
    message = initial_task_message,
    max_turns = 4)

print("------------------------------------------------------------------")
print("--- Conversation Ended (Multi-Model) ---")

In [ ]:
!pip install -q langchain_google_vertexai